# Classify Industries

## TODOS 

- top 10 klassen oder so
- preprocessing function einführen
    - `\n` weg
    - andere unnütze Zeichen wie `|` etc.
- remove pos einführen (siehe `clustering_whole_corpus`)
    - vielleicht mit Language identifier?
- ~~"country" als colum~~~~

## IDEEN

### Datensatzaufbereitung

- Übersetzung der Websites in einheitliche Sprache (z.b. Englisch)
- Andere Klassenlabels?
    - Erst allgemeinere Klassen und dann in diesen Klassen feiner klassifizieren?
        - Von: https://towardsdatascience.com/industrial-classification-of-websites-by-machine-learning-with-hands-on-python-3761b1b530f1
            - Technology, Office, & Education products website (Class_1)
            - Consumer products website (Class_2)
            - Industrial Tools and Hardware products website (Class_3)
    - seltene Klassenlables wegwerfen?

### HTML Klassifizierung


- Text zusammenfassen und dann klassifizieren? Dafür auch HTML-Tags verwenden?

- HTML Struktur verwenden, um vorher **Boilerplate Content** von Main Content zu entfernen:
    - Plain Text ist sehr noisy (viel unnötiges drin)
- Bestimmten Wörtern/Tags höhere Gewichtungen geben
    - Anchor Text (= klickbarer Text in einem Hyperlink)
        - alleine zu wenig Inhalt (QI, S. 12)
        - umliegende Wörter interessant! (QI, S. 12)
        - auch für Nachbar-Seiten-Ansatz
    - Title, Headers (QI, S. 12)
        - auch für Nachbar-Seiten-Ansatz
    - Keywords für Branchen
        ```python3  
        Class_1_keywords = ['Office', 'School', 'phone', 'Technology', 'Electronics', 'Cell', 'Business', 'Education', 'Classroom']
        
        Class_2_keywords = ['Restaurant', 'Hospitality', 'Tub', 'Drain', 'Pool', 'Filtration', 'Floor', 'Restroom', 'Consumer', 'Care', 'Bags', 'Disposables']
        
        Class_3_keywords = ['Pull', 'Lifts', 'Pneumatic', 'Emergency', 'Finishing', 'Hydraulic', 'Lockout', 'Towers', 'Drywall', 'Tools', 'Packaging', 'Measure', 'Tag ']
        ```
- NER mit **Tags** als weitere Tokens
- Features von "Nachbarseiten" verwenden
    - Hilfreich, da mehr Infos als "nur" Startseite
    - Fragen: 
        - Was sind Nachbarseiten, wie definieren?
            - Webgraph Webseiten?
            - Weitere Seiten des Unternehmens?
        - Wie viele Nachbarseiten?
        - Wieviel von den Nachbarseiten verwenden?
            - Ganze Seite?
            - text, title, heading, Metadaten?
    
- *Weiteres*:
    - Flat classification oder Hierarchical classification?
        - Flat: parallele Klassen
        - Hierarchical: hierarchische Klassen, bauen aufeinander auf
    - Nur nach bestimmten Keywords filtern? (das geht jedoch mehr Richtung PLAIN-Textclassification)
    - "implicit links": Seiten, die beide bei Suche von **Suchmaschine** erschienen sind und auf die beide der User geklickt hat (QI, S. 12) &rarr; nicht wirklich realisierbar


## Paper / Repos

- **Boilerplate Removal using a Neural Sequence Labeling Model** (2020): https://arxiv.org/pdf/2004.14294.pdf
    - Verbesserung von **Web2Text** &rarr; basiert nicht auf teuren, handgemachten Feature Engineering
    - <u>Hypothese</u>: "Our hypothesis is that the **order** of text blocks in a web page **encodes important information** about their type, i.e. content or boilerplate, as the placement is determined by the authoring style"
- **Web2Text: Deep Structured Boilerplate Removal** (2018): https://arxiv.org/pdf/1801.02607.pdf
- **Mozillas readability**: https://github.com/mozilla/readability
- **Webpage Classification based on Compound of Using HTML Features & URLFeatures and Features of Sibling Pages** (2010): https://www.researchgate.net/publication/220419545_Webpage_Classification_based_on_Compound_of_Using_HTML_Features_URL_Features_and_Features_of_Sibling_Pages
    - TODO
- **Web Page Classification: Features and Algorithms** (2009): https://www.cs.ucf.edu/~dcm/Teaching/COT4810-Fall%202012/Literature/WebPageClassification.pdf
    - S. 7: Using On-Page Features
        - GOLUB, ARDO (2005): title, headings, metadata, main text
    - TODO

## Tests

- Evaluation metric: **F1 Scores**
- TF-IDF Vectorizer
    - kein lowercase
    - stop words werden entfernt
    - keine max features
- Top $n$ classes = most frequent classes
- CLEAN HTML auch für Test Set (ansonsten unglaublich schlechte Accuracy und etwas sinnlos)


#### Label: `group_representatives`

| Experiment | KNN F1 (Precision) | LSVM F1 (Precision) |
| ---------- |:-----:| ----:|
| Plain Text (all samples) | **0.4295** (0.4774) | **0.6874** (0.7178) |
| Plain HTML (all samples) | **0.1474** (0.1954) | **0.5109** (0.6139) |
| | | |
| Plain Text ([DE] all samples) | **0.5224** (0.5563) | **0.7209** (0.7476) |
| Plain HTML ([DE] all samples) | **0.1515** (0.1915) | **0.5194** (0.6232) |
| Plain Text ([DE, EN] all samples) | **0.472** (0.5258) | **0.7012** (0.7262) |
| Plain HTML ([DE, EN] all samples) | **0.1466** (0.197) | **0.5166** (0.6211) |
| | | |
| Plain Text (all samples) (Top 10 classes) | **0.2059** (0.203) | **0.3024** (0.2721) |
| Plain HTML (all samples) (Top 10 classes) | **0.0943** (0.0995) | **0.2376** (0.2323) |
| Plain Text ([DE] all samples) (Top 10 classes) | **0.2212** (0.2036) | **0.3006** (0.2687) |
| Plain HTML ([DE] all samples) (Top 10 classes) | **0.089** (0.0912) | **0.2231** (0.2164) |
| | | |
| Clean HTML (all samples)  | **0.0612** (0.0912) | **0.5086** (0.6424) |
| Clean HTML ([DE] all samples)  | **0.0624** (0.0806) | **0.5579** (0.6607) |
| Clean HTML ([DE, EN] all samples)  | **0.065** (0.088) | **0.5354** (0.645) |

#### Label: `industry`

| Experiment | KNN F1 (Precision) | LSVM F1 (Precision) |
| ---------- |:-----:| ----:|
| Plain Text (all samples)  | **0.3788** (0.4416) | **0.6218** (0.6423) |
| Plain HTML (all samples)  | **0.1295** (0.1933) | **0.4589** (0.537) |
| | | |
| Plain Text ([DE] all samples)  | **0.4592** (0.5016) | **0.6446** (0.6591) |
| Plain HTML ([DE] all samples)  | **0.1285** (0.1949) | **0.4574** (0.5382) |

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# sklearn classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

# sklearn clustering
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# sklearn general
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder


from stop_words import get_stop_words
import ujson as json


import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from app.utils import clean_html_boilerplate

## Hyperparameters

In [3]:
DATA_DIR_PATH = "../data/"
LANG = "_DE"


INDUSTRIES_PATH_CSV = DATA_DIR_PATH + "industries.csv"
TRAIN_PATH_CSV = DATA_DIR_PATH + "train" + LANG + ".csv"
TEST_PATH_CSV = DATA_DIR_PATH + "test" + LANG + ".csv"

In [4]:
# "text" or "html"
TEXT_COL = "html"

# "group_representative", "group_representative_label", "industry", "industry_label" or "group"
CLASS_COL = "group_representative"
CLASS_NAMES = "group_representative_label"

MAX_DOCUMENT_FREQUENCY = 1.
MAX_FEATURES = 1000000
LOWERCASE = False
STOP_WORDS = get_stop_words("de")

# POS TAGGING
POS_TAGGING = False
POS_TAGS = ["NOUN"]
   

# USE TOP N CLASS LABELS
USE_TOP_LABELS = False
TOP_N_LABELS = 10


# SUBSAMPLING
SUBSAMPLING = False
N_SAMPLES = 10000000
USED_LANG = ["DE"] # "ALL" for no removal

# Load train csv

In [6]:
%%time
train = pd.read_csv(TRAIN_PATH_CSV)

CPU times: user 4.45 s, sys: 554 ms, total: 5 s
Wall time: 5 s


In [7]:
train.head(1)

,url,industry,industry_label,group,group_representative,html,text,source,country,group_representative_label
0,http://www.kswtech.com,135,Mechanical or Industrial Engineering,"cons, gov, man",135,"<html lang=""de""> <head> ...",KSW Elektro- und Industrieanlagenbau GmbH - KS...,xing,DE,Mechanical or Industrial Engineering


In [8]:
train.shape

(19842, 10)

In [10]:
len(list(train.industry.value_counts()))

30

## Some informations about the dataset

In [8]:
print("Most frequent countries:\n")
train.country.value_counts().head(5)

Most frequent countries:



DE    19842
Name: country, dtype: int64

In [9]:
text_percentage = train.apply(lambda row: len(row.text)/len(row.html), axis=1)

print(f"Average/mean share of actual/plain text of HTML: {np.round(np.mean(text_percentage), decimals=2)*100}%")

Average/mean share of actual/plain text of HTML: 19.0%


In [10]:
unique_classes = list(np.unique(train[CLASS_NAMES]))

print(f"{CLASS_NAMES} ({len(unique_classes)}): \n")
for idx, i in enumerate(unique_classes):
    print(str(idx+1)+". "+str(i), end="\t")

group_representative_label (21): 

1. Automotive	2. Construction	3. Consumer Goods	4. Financial Services	5. Human Resources	6. Information Technology and Services	7. Insurance	8. Legal Services	9. Leisure, Travel & Tourism	10. Logistics and Supply Chain	11. Machinery	12. Management Consulting	13. Marketing and Advertising	14. Mechanical or Industrial Engineering	15. Media Production	16. Medical Practice	17. Real Estate	18. Recreational Facilities and Services	19. Renewables & Environment	20. Telecommunications	21. Wholesale	

## SUBSAMPLING

- Only specific language (e.g. "DE")
- Only $n$ samples (e.g. 1000)
- Stratified sampling by industry col

In [11]:
if SUBSAMPLING:
    if USED_LANG[0] != "ALL":
        train = train[train.country.isin(USED_LANG)]
    if N_SAMPLES < train.shape[0]:
        max_samples = N_SAMPLES
    else:
        max_samples = train.shape[0]
    train = train.sample(n=max_samples, weights=CLASS_COL, random_state=1).reset_index(drop=True)
    
    
unique_sampled_classes = len(train[CLASS_COL].unique())
print("Count of classes (sampled train):", unique_sampled_classes)
print("Equal to original train?", unique_sampled_classes == len(unique_classes))
train.shape

Count of classes (sampled train): 21
Equal to original train? True


(19842, 10)

## USE TOP N LABELS

- only use top n classes

In [12]:
if USE_TOP_LABELS:
    top_n_classes = train[CLASS_COL].value_counts()[:TOP_N_LABELS].keys()
    train = train[train[CLASS_COL].isin(top_n_classes)]
    
    unique_classes = list(np.unique(train[CLASS_NAMES]))

    print(f"{CLASS_NAMES} ({len(unique_classes)}): \n")
    for idx, i in enumerate(unique_classes):
        print(str(idx+1)+". "+str(i), end="\t")
else:
    print("Using all labels.")

Using all labels.


# Data preprocessing (vectorizing, dimension reducing etc.)

- ignore terms with a document frequency > MAX_DOCUMENT_FREQUENCY (`max_df` in TF-IDF)

In [13]:
%%time

train_labels = train[CLASS_COL].values
unique_train_labels = list(np.unique(train[CLASS_COL]))
print("Count of unique classes in train set:", len(unique_train_labels))
print("Count of unique languages in train set:", len(np.unique(train["country"].values)))

Count of unique classes in train set: 21
Count of unique languages in train set: 1
CPU times: user 5.82 ms, sys: 705 µs, total: 6.53 ms
Wall time: 5.83 ms


### Remove tokens with POS-Tagging

In [14]:
%%time

if POS_TAGGING:
    train_text = remove_pos(train, pos_tags=POS_TAGS)
else:
    train_text = train[TEXT_COL]
    print("No POS TAGS are removed.\n")

No POS TAGS are removed.

CPU times: user 158 µs, sys: 46 µs, total: 204 µs
Wall time: 172 µs


### Vectorizing text

In [15]:
%%time

vectorizer = TfidfVectorizer(max_df=MAX_DOCUMENT_FREQUENCY,
                             lowercase=LOWERCASE,
                             max_features=MAX_FEATURES,
                             stop_words=STOP_WORDS)


vectorizer.fit(train_text)

train_vector = vectorizer.transform(train_text)

CPU times: user 1min 21s, sys: 369 ms, total: 1min 21s
Wall time: 1min 21s


# Test Dataset

In [16]:
%%time
test = pd.read_csv(TEST_PATH_CSV)

if SUBSAMPLING:
    if USED_LANG[0] != "ALL":
        test = test[test.country.isin(USED_LANG)]
    test = test.sample(n=test.shape[0], weights=CLASS_COL, random_state=1).reset_index(drop=True)
    

test_vector = vectorizer.transform(test[TEXT_COL].values)
test_labels = test[CLASS_COL].values

CPU times: user 11 s, sys: 145 ms, total: 11.2 s
Wall time: 11.2 s


# K-Nearest Neighbors

In [17]:
%%time
print("K-Nearest Neighbors CLF", "\n-------------------------")
# training
clf = KNeighborsClassifier()
clf.fit(train_vector, train_labels)

# prediction
train_preds = clf.predict(test_vector)

# evaluation
precision = precision_score(test_labels, train_preds, average="macro", zero_division=0)
recall = recall_score(test_labels, train_preds, average="macro", zero_division=0)
f1 = f1_score(test_labels, train_preds, average="macro", zero_division=0)
clf1_f1 = np.round(f1, decimals=4)
clf1_precision = np.round(precision, decimals=4)

print(np.round(precision, decimals=4), "\tPrecision")
print(np.round(recall, decimals=4), "\tRecall")
print(np.round(f1, decimals=4), "\tF1")
print()

clf_report = classification_report(test_labels, 
                                   train_preds, 
                                   target_names = np.unique(test[CLASS_NAMES]), 
                                   zero_division = 0)

K-Nearest Neighbors CLF 
-------------------------
0.1683 	Precision
0.1265 	Recall
0.13 	F1

CPU times: user 14.7 s, sys: 561 ms, total: 15.2 s
Wall time: 15.2 s


# LSVM

In [ ]:
%%time
print("LSVM CLF", "\n-------------------------")
# training
clf = LinearSVC(C = 1)
clf.fit(train_vector, train_labels)

# prediction
train_preds = clf.predict(test_vector)

# evaluation
precision = precision_score(test_labels, train_preds, average="macro", zero_division=0)
recall = recall_score(test_labels, train_preds, average="macro", zero_division=0)
f1 = f1_score(test_labels, train_preds, average="macro", zero_division=0)
clf2_f1 = np.round(f1, decimals=4)
clf2_precision = np.round(precision, decimals=4)

print(np.round(precision, decimals=4), "\tPrecision")
print(np.round(recall, decimals=4), "\tRecall")
print(np.round(f1, decimals=4), "\tF1")
print()

clf_report = classification_report(test_labels, 
                                   train_preds, 
                                   target_names = np.unique(test[CLASS_NAMES]),
                                   zero_division = 0)

LSVM CLF 
-------------------------


## Summary: Classification Results

In [ ]:
result = "| "

if TEXT_COL == "text":
    result += "Plain Text"
else:
    result += "HTML"
    
result += " ("
    
if SUBSAMPLING:
    result += "[" + ", ".join(USED_LANG) + "]"
    if N_SAMPLES < train.shape[0]:
        result += f" {N_SAMPLES} samples"
    else:
        result += " all samples"
else:
    result += "all samples"
    
result += ") "

if USE_TOP_LABELS:
    result += f"(Top {TOP_N_LABELS} classes)"
        
result += f" | **{clf1_f1}** ({clf1_precision}) | **{clf2_f1}** ({clf2_precision}) |"
print(CLASS_COL)
print()
print(result)

# Confusion Matrix

TODO: label und text names und so; allg. änderungen von oben hier ergänzen

In [ ]:
NORMALIZE_CM = True
INDUSTRY_TRESHOLD = 250
PLT_SCALING_FACTOR = 0.8

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

filtered_train = train.groupby(CLASS_COL).filter(lambda x: len(x)>INDUSTRY_TRESHOLD)
remaining_industries = filtered_train[CLASS_NAMES].drop_duplicates().tolist()


cnf_matrix = confusion_matrix(test_labels, train_preds)

classes = train[CLASS_COL].drop_duplicates().tolist()

cnf_df = pd.DataFrame(cnf_matrix, index=classes, columns=classes)
cnf_df = cnf_df[remaining_industries]
cnf_df = cnf_df.loc[remaining_industries]

In [ ]:
plt.figure(figsize=(10*PLT_SCALING_FACTOR, 8*PLT_SCALING_FACTOR))

if NORMALIZE_CM:
    normalized_cnf_df = cnf_df.astype('float') / cnf_df.sum(axis=1)[:, np.newaxis]
    sns.heatmap(normalized_cnf_df, annot=True, cmap=sns.color_palette("Blues"), fmt='.2f')
else:
    sns.heatmap(cnf_df, annot=True, cmap=sns.color_palette("Blues"), fmt='g')
plt.tight_layout()